# How FEMR Works + Toy Example

This notebook will go over the fundamentals of FEMR, including the three core classes of `Patient`, `Event` and `PatientDatabase`.

Everything in **FEMR** revolves around processing patients and their clinical data. This is done through these three fundamental classes.

Before going through this tutorial, please make sure you've installed **FEMR** as detailed in the [README.md](https://github.com/som-shahlab/femr)

In [1]:
import datetime
import femr

#### Patients

FEMR represents every patient with the `Patient` class. 

A `Patient` object contains the following two attributes:

* `patient_id` (str): a unique identifier for the patient
* `events` (list): a list of `Event` objects associated with that patient

The definition of the `Patient` class can be [found here](https://github.com/som-shahlab/femr/blob/main/src/femr/__init__.py#L11)

#### Events

FEMR represents clinical events with the `Event` object.

An `Event` object can contain any number of arbitrary attributes, but it must have at least the following two attributes:

* `start` (datetime.datetime): the start time of the event
* `code` (int): the code that FEMR associates with events of the same type

The definition of the `Event` class can be [found here](https://github.com/som-shahlab/femr/blob/main/src/femr/__init__.py#L22)


#### PatientDatabase

FEMR stores patients and events in the `PatientDatabase` object.

A `PatientDatabase` is a Python map between patient ids and `Patient` objects. It also contains a variety of other utilities, such as ontology functionality.

`PatientDatabase`s can be created in two ways, either directly from OMOP 5.4 data (the recommended route), or through the simple FEMR format.

See tutorial 2a for the OMOP route and tutorial 2b for the simple FEMR format route.

Below, we'll create some events for some fictional patients.

In [2]:
events = [
    # This event contains the bare minimum attributes -- start and code.
    femr.Event(
        start=datetime.datetime(2010, 1, 5),
        code=2,
    ),
    # This event contains a couple custom attributes -- value and source_table.
    femr.Event(
        start=datetime.datetime(2010, 1, 3, hour=10, minute=45),
        code=2,
        value="test_value",
        source_table=None,
    ),
    # This event contains even more attributes.
    femr.Event(
        start=datetime.datetime(2010, 1, 3, hour=10, minute=30),
        code=0,
        value=34.0,
        source_table='visit',
        extra_attr=True,
    ),
]

Now that we have our events, let's create a couple fictional patients.

In [3]:
patients = [
    # Let's assign each patient a subset of our events for example purposes.
    femr.Patient(patient_id=0, events=events[:2]),
    femr.Patient(patient_id=1, events=events[1:]),
    femr.Patient(patient_id=10, events=events),
]

# Lets print out the events of one patient to see what they look like.
# Note that attributes associated with `None` (i.e. `source_table` in the 
# second event) won't be printed out, but they are still tracked internally by femr.
patients[0].events

[Event(start=2010-01-05 00:00:00, code=2, value=None),
 Event(start=2010-01-03 10:45:00, code=2, value=test_value)]

We can access a specific event's attributes as follows. Note that accessing an attribute that wasn't defined on an event will return `None` by default (rather than raise an exception).

In [4]:
for patient in patients:
    print(f"Patient {patient.patient_id}:")
    for idx, event in enumerate(patient.events):
        print(f"    Event #{idx}:")
        print(f"       Start = {event.start} | Code = {event.code} | Value = {event.value} | Source Table = {event.source_table} | Extra Attr = {event.extra_attr}")

Patient 0:
    Event #0:
       Start = 2010-01-05 00:00:00 | Code = 2 | Value = None | Source Table = None | Extra Attr = None
    Event #1:
       Start = 2010-01-03 10:45:00 | Code = 2 | Value = test_value | Source Table = None | Extra Attr = None
Patient 1:
    Event #0:
       Start = 2010-01-03 10:45:00 | Code = 2 | Value = test_value | Source Table = None | Extra Attr = None
    Event #1:
       Start = 2010-01-03 10:30:00 | Code = 0 | Value = 34.0 | Source Table = visit | Extra Attr = True
Patient 10:
    Event #0:
       Start = 2010-01-05 00:00:00 | Code = 2 | Value = None | Source Table = None | Extra Attr = None
    Event #1:
       Start = 2010-01-03 10:45:00 | Code = 2 | Value = test_value | Source Table = None | Extra Attr = None
    Event #2:
       Start = 2010-01-03 10:30:00 | Code = 0 | Value = 34.0 | Source Table = visit | Extra Attr = True
